In [ ]:
!pip install pandas pdfplumber tqdm

In [2]:
import os
import pdfplumber
import pandas as pd
from tqdm import tqdm
import math


def extract_text_from_pdf(pdf_path):
    text = []
    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                page_text = page.extract_text()
                if page_text:
                    text.append(page_text)
        return "\n".join(text)
    except Exception as e:
        print(f"Ошибка при обработке {pdf_path}: {e}")
        return ""


def process_pdfs_in_folder(folder_path):
    pdf_files = [f for f in os.listdir(folder_path) if f.endswith(".pdf")]
    total_files = len(pdf_files)
    chunk_size = math.ceil(total_files / 4)

    for stage in range(4):
        start_idx = stage * chunk_size
        end_idx = min((stage + 1) * chunk_size, total_files)
        chunk_files = pdf_files[start_idx:end_idx]
        data = []

        print(f"Этап {stage + 1}: обработка файлов {start_idx + 1}-{end_idx} из {total_files}")

        for filename in tqdm(chunk_files, desc=f"Этап {stage + 1}", unit="файл"):
            file_path = os.path.join(folder_path, filename)
            full_text = extract_text_from_pdf(file_path)
            data.append([filename, full_text])

        stage_output_csv = output_csv.replace(".csv", f"_stage{stage + 1}.csv")
        df = pd.DataFrame(data, columns=["Название файла", "Текст"])
        df.to_csv(stage_output_csv, index=False, encoding='utf-8')
        print(f"Этап {stage + 1} завершен. Результат сохранен в {stage_output_csv}")



In [5]:
folder_path = "../reports"
output_csv = "результат.csv"
process_pdfs_in_folder(folder_path)

Этап 1: обработка файлов 1-367 из 1467


Этап 1:   0%|          | 1/367 [00:03<22:45,  3.73s/файл]


KeyboardInterrupt: 

In [7]:
def combine_csv_files(file_list, output_csv):
    """Объединяет несколько CSV файлов в один."""
    combined_df = pd.concat([pd.read_csv(f) for f in file_list], ignore_index=True)
    combined_df.to_csv(output_csv, index=False, encoding='utf-8')
    print(f"Все этапы завершены. Итоговый файл сохранен в {output_csv}")

file_list = ['результат_stage1.csv','результат_stage2.csv','результат_stage3.csv','результат_stage4.csv']

combine_csv_files(file_list, output_csv)

Все этапы завершены. Итоговый файл сохранен в результат.csv


In [3]:
rspp_data = pd.read_csv('результат.csv')

In [4]:
rspp_clean = rspp_data.dropna()

## По параграфам

In [7]:
!pip install pymorphy3 nltk

  Obtaining dependency information for nltk from https://files.pythonhosted.org/packages/4d/66/7d9e26593edda06e8cb531874633f7c2372279c3b0f46235539fe546df8b/nltk-3.9.1-py3-none-any.whl.metadata
  Obtaining dependency information for click from https://files.pythonhosted.org/packages/7e/d4/7ebdbd03970677812aac39c869717059dbb71a4cfc033ca6e5221787892c/click-8.1.8-py3-none-any.whl.metadata
  Obtaining dependency information for joblib from https://files.pythonhosted.org/packages/91/29/df4b9b42f2be0b623cbd5e2140cafcaa2bef0759a00b7b70104dcfe2fb51/joblib-1.4.2-py3-none-any.whl.metadata
  Obtaining dependency information for regex>=2021.8.3 from https://files.pythonhosted.org/packages/80/32/763a6cc01d21fb3819227a1cc3f60fd251c13c37c27a73b8ff4315433a8e/regex-2024.11.6-cp311-cp311-win_amd64.whl.metadata
     ---------------------------------------- 0.0/41.5 kB ? eta -:--:--
     ---------------------------- --------- 30.7/41.5 kB 660.6 kB/s eta 0:00:01
     -------------------------------------- 4


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
from pymorphy3 import MorphAnalyzer
import re
morph = MorphAnalyzer()

In [9]:
# Загрузка стоп-слов
nltk.download("stopwords")
stopwords_ru = set(stopwords.words("russian"))

# Регулярные выражения
RE_DERIVED = re.compile(r"\w+( -|- |-|! - )\w+")
RE_RUSSIAN_TEXT = re.compile(r"[а-яА-Яa-zA-Z0-9\-]+")

# Инициализация морфологического анализатора
morph = MorphAnalyzer()



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\waksi\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [10]:
def preprocess_text(text):
    paragraphs = text.split("\n")
    processed_paragraphs = []
    
    for i, para in enumerate(paragraphs):
        words = RE_RUSSIAN_TEXT.findall(para)
        words = [morph.parse(word.lower())[0].normal_form for word in words if word.lower() not in stopwords_ru]
        para_processed = " ".join(words)
        processed_paragraphs.append((i + 1, para_processed))
    
    return processed_paragraphs

def process_dataframe(df, stage=4):
    if "Текст" not in df.columns:
        raise ValueError("В DataFrame должен быть столбец 'Текст'")
    
    total_rows = len(df)
    chunk_size = total_rows // stage + (total_rows % stage > 0)
    all_data = []
    
    for i in range(stage):
        start = i * chunk_size
        end = min((i + 1) * chunk_size, total_rows)
        print(f"Этап {i+1}/{stage}: обработка строк {start+1} - {end}")
        
        for index, row in tqdm(df.iloc[start:end].iterrows(), total=end-start, desc=f"Этап {i+1}"):
            paragraphs = preprocess_text(row["Текст"])
            temp_df = pd.DataFrame(paragraphs, columns=["Номер параграфа", "Параграф"])
            temp_df.insert(0, "Название файла", row.get("Название файла", f"Строка_{index+1}"))
            all_data.append(temp_df)
    
    return pd.concat(all_data, ignore_index=True)

In [11]:
df_result = process_dataframe(rspp_clean)

Этап 1/4: обработка строк 1 - 332


Этап 1: 100%|██████████| 332/332 [07:02<00:00,  1.27s/it]


Этап 2/4: обработка строк 333 - 664


Этап 2: 100%|██████████| 332/332 [11:14<00:00,  2.03s/it]


Этап 3/4: обработка строк 665 - 996


Этап 3: 100%|██████████| 332/332 [16:56<00:00,  3.06s/it]


Этап 4/4: обработка строк 997 - 1326


Этап 4: 100%|██████████| 330/330 [17:26<00:00,  3.17s/it]


In [17]:
df_result.tail()


,Название файла,Номер параграфа,Параграф
6782007,2023_Яндекс.pdf,4236,яндекс деловой партн ры яндекс деловой партн р...
6782008,2023_Яндекс.pdf,4237,яндекс деловой партн ры
6782009,2023_Яндекс.pdf,4238,акционер инвестор акционер инвестор акционер и...
6782010,2023_Яндекс.pdf,4239,акционер инвестор
6782011,2023_Яндекс.pdf,4240,97


In [19]:
df_result.to_csv('paragraphs.csv')